# Data Preparation

In [ ]:
# dataset_path = '/Users/gitaayusalsabila/Documents/0thesis/code/sandbox/dataset/'
dataset_path = '/notebooks/dataset/'

In [ ]:
def multivariate_simulate(datapath, n_samples=200,n_time=2,n_views=4):
    # Note that changing the node count is not provided right now, since we use correlation matrix
    # and the mean values of connectivities from real data and it is for 35 nodes.
    
    # Import all required statistical information.
    allstats = np.load(datapath + "stats/REAL_TIME_DIFF.npy") # Connectivity mean values of LH. You can also try with RH.
    allcorrs = np.load(datapath + "stats/REAL_TIME_DIFF.npy") # Correlation matrix in LH. You can also try with RH.
    all_diffs = np.load(datapath + "stats/REAL_TIME_DIFF.npy") # This is an overall representation of time differences in both (LH and RH) datasets.
    
    times = []
    for t in range(n_time):
        views = []
        for v in range(n_views):
            # Note that we randomly assign a new random state to ensure it will generate a different dataset at each run.
            # Generate data with the correlations and mean values at the current timepoint.
            if t < 2:
                connectomic_means = allstats[t,v]
                data = multivariate_normal.rvs(connectomic_means,allcorrs[t,v],n_samples,random_state=randint(1,9999))
            # If the requested timepoints are more than we have in real data, use the correlation information from the last timepoint.
            else:
                connectomic_means = allstats[-1,v]
                data = multivariate_normal.rvs(connectomic_means,allcorrs[-1,v],n_samples,random_state=randint(1,9999))

            adj = []
            for idx, sample in enumerate(data):
                # Create adjacency matrix.
                matrix = antiVectorize(sample,35)
                # Perturb the real time difference with nonlinear tanh function.
                noise = np.tanh( t / n_time )
                # Further timepoints will have more significant difference from the baseline (t=6 >> t=1).
                matrix = matrix + all_diffs[:,:,v] * ( noise + 1 )
                adj.append(matrix)
            views.append(np.array(adj))

        times.append(np.array(views))
    
    alldata=np.array(times)
    alldata = np.transpose(alldata,(2,0,3,4,1))
    return alldata 

def prepare_data(datapath, new_data=False, n_samples=200, n_times=6):
    # Note that data with 200 samples and 6 timepoints is very large (5.8M data points),
    # check your GPU memory to make sure there is enough space to allocate. If not, try:
    # - to reduce dataset size by changing n_samples or n_times.
    # - on CPU (this will allocate memory on RAM) --> This might work for example if you have 1GB GPU memory but 16GB RAM.
    # - on another computer with a better NVIDIA graphics card. --> 2GB GPU memory will not be enough for 5.8M data.
    try:
        if new_data:
            samples = multivariate_simulate(datapath, n_samples, n_times)
            np.save(datapath + 'simulated_adj.npy',samples)
        else:
            samples = np.load(datapath + 'simulated_adj.npy')
    except:
        samples = multivariate_simulate(datapath, n_samples, n_times)
        np.save(datapath + 'simulated_adj.npy',samples)
    return samples

simulated_data = prepare_data(dataset_path, new_data=False, n_samples=100, n_times=3)
simulated_data.shape

In [ ]:
slim160 = np.load(dataset_path + 'slim160_adj.npy')
print(slim160.shape)

slim268 = np.load(dataset_path + 'slim268_adj.npy')
print(slim268.shape)

In [ ]:
def data_cleansing(dataset):
    # Replace negative values with 0
    dataset[dataset < 0] = 0
    
    # Replace NaN values with 0
    dataset = np.nan_to_num(dataset, nan=0)
    
    return dataset

def check_and_drop_invalid_graphs(graph_dataset):
    """
    Check that all graphs in the dataset have more than 0 edges and drop graphs with 0 edges in any timepoint or dimension.
    
    Parameters:
    graph_dataset (np.ndarray): The input graph dataset with shape [g, t, n, n, d] or [g, t, n, n]
    
    Returns:
    np.ndarray: The cleaned dataset with invalid graphs removed
    """
    
    # Check the shape of the dataset to determine if it has multiple dimensions
    if len(graph_dataset.shape) == 5:
        num_graphs, num_timepoints, num_nodes, _, num_dimensions = graph_dataset.shape
    else:
        num_graphs, num_timepoints, num_nodes, _ = graph_dataset.shape
        num_dimensions = 1
    
    valid_graphs = []

    for i in range(num_graphs):
        is_valid = True
        for t in range(num_timepoints):
            for d in range(num_dimensions):
                if num_dimensions > 1:
                    adj_matrix = graph_dataset[i, t, :, :, d]
                else:
                    adj_matrix = graph_dataset[i, t, :, :]
                
                num_edges = np.sum(adj_matrix > 0)
                if num_edges == 0:
                    is_valid = False
                    break
            if not is_valid:
                break
        
        if is_valid:
            valid_graphs.append(i)
    
    if num_dimensions > 1:
        cleaned_dataset = graph_dataset[valid_graphs, :, :, :, :]
    else:
        cleaned_dataset = graph_dataset[valid_graphs, :, :, :]
    
    return cleaned_dataset

In [ ]:
simulated_cleaned = data_cleansing(simulated_data)
simulated_cleaned = check_and_drop_invalid_graphs(simulated_data)
print('simulated data shape:',simulated_cleaned.shape)

slim160_cleaned = data_cleansing(slim160)
slim160_cleaned = check_and_drop_invalid_graphs(slim160_cleaned)
print('slim160 shape:',slim160_cleaned.shape)

slim268_cleaned = data_cleansing(slim268)
slim268_cleaned = check_and_drop_invalid_graphs(slim268_cleaned)
print('slim268 shape:',slim268_cleaned.shape)

In [ ]:
## Laplacian Encoding
simulated_laplacian_features = np.load(dataset_path + 'simulated_laplacian_features.npy') 
slim160_laplacian_features = np.load(dataset_path + 'slim160_laplacian_features.npy') 
slim268_laplacian_features = np.load(dataset_path + 'slim268_laplacian_features.npy') 

## Degree Encoding
simulated_degree_features = np.load(dataset_path + 'simulated_degree_features.npy') 
slim160_degree_features = np.load(dataset_path + 'slim160_degree_features.npy') 
slim268_degree_features = np.load(dataset_path + 'slim268_degree_features.npy') 

## Node2Vec


# RGBM

In [ ]:
def eucledian_distance(x):
  repeated_out = x.repeat(35,1,1)
  repeated_t = torch.transpose(repeated_out, 0, 1)
  diff = torch.abs(repeated_out - repeated_t)
  return torch.sum(diff, 2)

def frobenious_distance(test_sample,predicted):
  diff = torch.abs(test_sample - predicted)
  dif = diff*diff
  sum_of_all = diff.sum()
  d = torch.sqrt(sum_of_all)
  return d

def create_edge_index_attribute(adj_matrix):
    rows, cols = adj_matrix.shape[0], adj_matrix.shape[1]
    edge_index = torch.zeros((2, rows * cols), dtype=torch.long, device='cuda')
    edge_attr = torch.zeros((rows * cols, 1), dtype=torch.float, device='cuda')
    counter = 0

    for src, attrs in enumerate(adj_matrix):
        for dest, attr in enumerate(attrs):
            edge_index[0][counter], edge_index[1][counter] = src, dest
            edge_attr[counter] = attr
            counter += 1

    return edge_index, edge_attr, rows, cols

In [ ]:
class RNNCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(RNNCell, self).__init__()
        self.weight = nn.Linear(input_dim, hidden_dim, bias=True)
        self.weight_h = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.out = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.tanh = Tanh()
    
    def forward(self,x):
        global hidden_state
        h = hidden_state
        y = self.tanh(self.weight(x) + self.weight_h(h))
        hidden_state = y.detach()
        return y


class RBGM(nn.Module):
    def __init__(self):
        super(RBGM, self).__init__()
        self.rnn = nn.Sequential(RNNCell(1,1225), ReLU())
        self.gnn_conv = NNConv(35, 35, self.rnn, aggr='mean', root_weight=True, bias = True)
        
    def forward(self, data):
        edge_index, edge_attr, _, _ = create_edge_index_attribute(data)
        x1 = F.relu(self.gnn_conv(data, edge_index, edge_attr))
        x1 = eucledian_distance(x1)
        return x1

    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

In [ ]:
rbgm_simulated_dom1 = GCRN(input_dim, feature_dim, latent_dim, encoder_hidden_dims, decoder_hidden_dims, num_rec_layers, 'GAT', 'GCN')

print(rbgm_simulated_dom1)
print(f"Total number of trainable parameters: {(rbgm_simulated_dom1.count_parameters())*2}\n")

In [ ]:
mael = torch.nn.L1Loss().to(device)
tp = torch.nn.MSELoss().to(device)

In [ ]:
def train_rbgm(model, train_adj, num_epochs=100, lr=0.001, save_path='models/rgbm_model.pth'):
                    
    model_1 = rbgm_simulated_dom1
    model_2 = rbgm_simulated_dom1
    optimizer_1 = torch.optim.Adam(model_1.parameters(), lr = lr)
    optimizer_2 = torch.optim.Adam(model_2.parameters(), lr = lr)

    training_loss = []

    epoch_time = []
    cpu_usage = []
    memory_usage = []
    
    images = []

    for epoch in range(num_epochs):
        set_seed(42)
        # Will be used for reporting

        train_loss_t1, train_loss_t2 = 0.0, 0.0 
        mae_loss_eval_t1, mae_loss_eval_t2 = 0.0, 0.0
        tp_loss_1, tp_loss_2, tr_loss_1, tr_loss_2 = 0.0, 0.0, 0.0, 0.0
        model_1.train()
        model_2.train()
        for i, data in enumerate(h_data_train_loader):
            
            
            #Time Point 1
            data = data.to(device)
            optimizer_1.zero_grad()
            out_1 = model_1(data.x)
            
           
            # Topological Loss
            
            tp_1 = tp(out_1.sum(dim=-1), data.y.sum(dim=-1))
            tp_loss_1 += tp_1.item()
            
            #MAE Loss
            loss_1 = mael(out_1, data.y)
            train_loss_t1 += loss_1.item()
            
            total_loss_1 = loss_1 + opt.tp_coef * tp_1
            tr_loss_1 += total_loss_1.item()
            total_loss_1.backward()
            optimizer_1.step()
            
            #Time Point 2
            
            optimizer_2.zero_grad()
            out_2 = model_2(data.y)
            

            # Topological Loss
           
            tp_2 = tp(out_2.sum(dim=-1), data.y2.sum(dim=-1))
            tp_loss_2 += tp_2.item()
            
            
            #MAE Loss
            loss_2 = mael(out_2, data.y2)
            train_loss_t2 += loss_2.item()
            
            total_loss_2 = loss_2 + opt.tp_coef * tp_2
            tr_loss_2 += total_loss_2.item()
            total_loss_2.backward()
            optimizer_2.step()
            
        print(f'Epoch [{epoch + 1}/{num_epochs}]')
        print(f'[Train] Loss T1 : {train_loss_t1 / total_step:.5f}, Loss T2 : {train_loss_t2 / total_step:.5f} ')
        print(f'[Train] TP Loss T1: {tp_loss_1 / total_step:.5f}, TP Loss T2 : {tp_loss_2 / total_step:.5f}')
        print(f'[Train] Total Loss T1: {tr_loss_1 / total_step:.5f}, Total Loss T2 : {tr_loss_2 / total_step:.5f}' )
        mae_loss_train_t1.append(train_loss_t1 / total_step)
        mae_loss_train_t2.append(train_loss_t2 / total_step)
        tp_loss_train_1.append(tp_loss_1 / total_step)
        tp_loss_train_2.append(tp_loss_2 / total_step)
        train_total_loss_1.append(tr_loss_1 / total_step)
        train_total_loss_2.append(tr_loss_2 / total_step)
        
    
    tic1 = timeit.default_timer()
    timer(tic0,tic1)

# EvoNet

# TS-Network